| Class | Collaborator | Responsibilities |
|---|---|---|
| Handle | — | • A handle identifies a source of indication events in an operating system<br>• Handle can queue up indication events |
| Handle Set | — | • A handle set is a collection of handles |
| Synchronous Event Demultiplexer | • Handle<br>• Handle Set | • Can block waiting for indication events to occur on a handle set<br>• Indicates an operation can be initiated on a handle without blocking |

| Class | Collaborator | Responsibilities |
|---|---|---|
| Event Handler | Handle | • Defines and interface for processing events that occur on a handle |
| Concrete Event handler | Handle | • Processes an indication event on a handle in an application specific manner <br> • Defines an application service |

## Implementazione

I partecipanti sono:
1. *Demultiplexing / dispatching infrastructure* Qui faccio tutto il lavoro generico e application-independent (la macchineria generica) legata a eventi e il dispacciamento degli eventi
2. *Application components* qui ci metto gli event handler che "incarnano" la application logic

### Event handler interface
Qui ci metto la classse e gli hook methods (qui posso usare il truccheto del Template Method). 
Prima però:
#### Definisci il tipo di dispatching target
Ci sono due tipi di target che io posso richiamare:
1. Event Handler Objects: qui uso un oggetto ed è comodo in quanto posso specializzarlo via ereditarità ed incapsulare il tutto
2. Event Handler Functions: qui uso più lambda e ci passo funzioni tramite callback o delegate (più leggero e "sganciato") 
In questo esempio uso la variante a **oggetti**.

#### Definisci la strategia di dispacciamento dell'interfaccia di event handling
Come si parlano (ceedono il controllo) gli event handler e il reactor che si dispacciano? Possono farlo tramite invocazione di metodo

In particolare nell'esempio io voglio passare il tipo di evento al event handler così che non tipizzo ogni event handler sul tipo di evento

In [1]:
#include <sys/select.h>
#include <sys/time.h>

#ifndef EVENT_TYPES_DEFINED
using HANDLE = int;
using Time_Value = timeval;

typedef unsigned int Event_Type;
// Types of indication events.
enum {
   READ_EVENT = 01, // ACCEPT_EVENT aliases READ_EVENT
   ACCEPT_EVENT = 01, // due to <select> semantics.
   WRITE_EVENT = 02,
   TIMEOUT_EVENT = 04,
   SIGNAL_EVENT = 010,
   CLOSE_EVENT = 020
   // These values are powers of two so
   // their bits can be "or'd" together efficiently.
};
#define EVENT_TYPES_DEFINED 1
#endif


Poi implementiamo la `Event_Handler` class. Qui uso metodi virtuali perché voglio che vengano risolti a runtime e agganciati al caller (figlio) corretto. Immagina se event1 fosse gestito dal handler del event2!



##### Single-Method
*Single-method* dispatch strategy: qui uso una singola invocazione del metodo. Il problema di questo è che incoraggia a usare uno `switch/case` statement per scegliere sui diversi tipi di eventi da dover gestire e il tipo di handler da invocare. Di base non è sbagliato, ma ogni votla che devi aggiungere un tipo di evento devi ricordarti di andare qui e gestirlo!

In [2]:
class Event_Handler { // Single-method interface.
public:
   // Hook method dispatched by <Reactor> to handle
   // events of a particular type.
   virtual void handle_event (HANDLE handle, Event_Type et) = 0;
   // Hook method that returns the I/O <HANDLE>.
   virtual HANDLE get_handle () const = 0;
protected:
   // Virtual destructor is protected to ensure
   // dynamic allocation.
   virtual ~Event_Handler () = default;
};


##### Multi-Method
*Double/Multi-method* dispatch strategy: invoco multiple funzioni. Avendo più tipi di funzioni a questo punto aggancio al reacotr la gestione e il disptching di quello corretto. Qui ci devo pensare più a lungo eprché devo capire bene cosa mi servirà in futuro altrimenti rischio di incorrere in un overloading dell'interfaccia e diventa un blob ingestibile (code smell della violazione del single-responsibility)

In [3]:
class Event_Handler_Multi_Method {
public:
   // Hook methods dispatched by a <Reactor> to handle
   // particular types of events.
   virtual void handle_input (HANDLE handle) = 0;
   virtual void handle_output (HANDLE handle) = 0;
   virtual void handle_timeout (const Time_Value &) = 0;
   virtual void handle_close (HANDLE handle,
                              Event_Type et) = 0;
   // Hook method that returns the I/O <HANDLE>.
   virtual HANDLE get_handle () const = 0;
   virtual ~Event_Handler_Multi_Method () = default;
};


### Reactor

Definisci l'interfaccia del `Reactor`. Cosa deve fare il `Reactor`? Principalmente permettere ai vari event handlers di registrare/rimuovere i loro metodi da far invocare. Questo verrà fatto attraverso il metodo `register_handler()`.  Il `Reactor` avrà un ciclo di vita unico e legagto all'applicazione e quindi lo dichiariamo come `Singleton` (in questo esempio lasciamo perdere per semplicità). Un'altra cosa è prevedere diversi tipi di `Reactor`, magari ognuno tipizzato per un certo tipo di sistema operativo o strategia... Per far questo usiamo un Bridge Pattern.
Ecco il padre

In [4]:
class Reactor_Implementation {
public:
   virtual ~Reactor_Implementation () = default;
   virtual void register_handler
      (Event_Handler *event_handler, Event_Type event_type) = 0;
};

class Reactor {
public:
   // Methods that register and remove <Event_Handler>s
   // of particular <Event_Type>s on a <HANDLE>.
   virtual void register_handler
      (Event_Handler *eh, Event_Type et) = 0;
   virtual void register_handler
      (HANDLE h, Event_Handler *eh, Event_Type et) = 0;
   virtual void remove_handler
      (Event_Handler *eh, Event_Type et) = 0;
   virtual void remove_handler
      (HANDLE h, Event_Type et) = 0;

   // Entry point into the reactive event loop. The
   // <timeout> can bound time waiting for events.
   void handle_events (Time_Value *timeout = 0);
   // Define a singleton access point.
   static Reactor *instance ();
private:
   // Use the Bridge pattern to hold a pointer to
   // the <Reactor_Implementation>.
   Reactor_Implementation *reactor_impl_ = 0;
};


#### Approfondimento: signatures del register_handler()

Questo metodo ha implicazioni importanti sul `Reactor` e può essere visto in 2 varianti:
1. *Two parameters* qui un parametro formale identifica il tipo di indication event per cui ho un handler registrato. Questo tipo usa un *double-dispatch* strategy per ottenere un handle. Per farlo uso il callback `get_handle()` -> impossibile associare per sbaglio il handler sbagliato a un evento.

Esempio con double-dispatch di una variante concreta (per forza...) di `Reactor`:

In [5]:
class Select_Reactor_Implementation : public Reactor_Implementation {
public:
   void register_handler
      (Event_Handler *event_handler,
      Event_Type event_type) override {
      // Double-dispatch to obtain the <HANDLE>.
      HANDLE handle = event_handler->get_handle ();
      (void)handle;
      (void)event_type;
   }
};


2. *Three parameters* qui faccio overloading aggiungendo un parametro in cui passo il handle direttamente. Qui il event handler viene usato come fosse una scatola "stateless" (simile a una library function) in cui passo tutto. 

#### Approfondimento: remove_handler()
Anche qui esattamente come per il register poso decidere di passare il `handle` alla funzione o farlo gestire direttamente al event handler che avrà "in pancia" il suo handle.

### Entry Point

Il `Reactor` definisce il entry point con la funzione `handle_events()`. Questo metodo viene usato dalle applicazioni che vogliono eseguire il proprio event loop. In particolare questo invocherà il synchronous event demultiplexer che aspetterà l'arrivo di event nei "punti di contatto" (NIC, ....). Successivamente, quando un handle è pronto (cioè è arrivato un evento e il OS lo ha incodato) la funzione esce e dal loop "bloccante" e va nella sezione del event demultiplexing con la lista dei handle pronti per esssere serviti e uno alla volta verranno serviti triggerando il relativo event handler. 

### Implemento l'interfaccia del Reactor

Ci sono 4 stepnel'implementazione del Reactor:

### Implementation hierarchy

La versione di Reactor sviluppat qui delega tutto il "circo" relativo allo smistamento (demultiplexing) e dispatching al Reactor (non ho altre classi). La cosa buona di questa strategia è che posso creare divere varianti di Reactor. Le diverse varianti poi e faccio usare alle mie app grazie a un Bridge pattern. Pensiamo al caso in cui un'app debba girare in diversi sistemi operativi, avrà a disposizione diversi Reactor e diversi synchronous Even Demuliplexers (select, epoll, Windows I/O Completion ports...)

### Synchronous Event Demultiplexer

Questo esempio girerà Linux e quindi usiamo `select` ome synchronous event demultiplexer. Questa funzione mi libera dal fardello di dover stare a vedre quale socket sia pronta.
``` c++
    int select (u_int max_handle_plus_1,
       fd_set *read_fds, fd_set *write_fds,
       fd_set *except_fds,timeval *timeout);
```

### Demultiplexing table

Devo avere un'associazione fra evento, event handler e handle, questa associazione risiede nella Event Demultiplexing Table <handle, event handler, indication event types>. Poiché memorizzo in una hash map ottengo una velocità di ricerca circa dell'ordine O(1). Ecco un esempio di implementazione:


In [ ]:
// Safety cell: se esegui da qui in giu, i simboli evento sono comunque disponibili.
#ifndef EVENT_TYPES_DEFINED
#include <sys/select.h>
#include <sys/time.h>
using HANDLE = int;
using Time_Value = timeval;
typedef unsigned int Event_Type;
enum {
   READ_EVENT = 01,
   ACCEPT_EVENT = 01,
   WRITE_EVENT = 02,
   TIMEOUT_EVENT = 04,
   SIGNAL_EVENT = 010,
   CLOSE_EVENT = 020
};
#define EVENT_TYPES_DEFINED 1
#endif


In [6]:
#include <sys/select.h>

class Demux_Table {
public:
   // Local fallback constants: this cell works even if prior event cells were not run.
   enum {
      READ_EVENT = 01,
      WRITE_EVENT = 02,
      CLOSE_EVENT = 020
   };

   struct Tuple {
      // Pointer to <Event_Handler> that processes
      // the indication events arriving on the handle.
      Event_Handler *event_handler_;
      // Bit-mask that tracks which types of indication
      // events <Event_Handler> is registered for.
      Event_Type event_type_;
   };

   // Converti <Tuple> array to <fd_set>s.
   void convert_to_fd_sets
      (fd_set &read_fds, fd_set &write_fds, fd_set &except_fds) {
      FD_ZERO (&read_fds);
      FD_ZERO (&write_fds);
      FD_ZERO (&except_fds);

      for (int h = 0; h < FD_SETSIZE; ++h) {
         Tuple &tuple = table_[h];
         if (tuple.event_handler_ == 0)
            continue;

         if (tuple.event_type_ & READ_EVENT)
            FD_SET (h, &read_fds);
         if (tuple.event_type_ & WRITE_EVENT)
            FD_SET (h, &write_fds);
         if (tuple.event_type_ & CLOSE_EVENT)
            FD_SET (h, &except_fds);
      }
   }

   // Table of <Tuple>s indexed by Handle values.
   // The macro FD_SETSIZE is typically defined in
   // <sys/socket.h>/<sys/select.h> system headers.
   Tuple table_[FD_SETSIZE] = {};
};


In nginx non ho una event demultiplex table ma è distribuito, nel senso che ogni evento ha in pancia il suo event handler e ogni sorgente (app interessata) manifesta l'interesse a un particolare evento creando un evento, mettendoci i puntatori a funzioni e incodandolo nel Reactor queue.

### Implementazione concreta Reactor

In questo caso poiché ho detto che avrei usato il select come Event Demultiplex allora creo il SelectConcreteReactor
``` c++
    class Select_Reactor_Implementation : public Reactor_Implementation {
      public:
```

Triggero tutto quanto cioè il entry point del mio reacotr col metodo `handle_events()`
``` c++
    void Select_Reactor_Implementation::handle_events(Time_Value *timeout = 0) {
```
Per prima cosa prima converto il `Demux_Tabl handle` in `fd_set` handle set che passo alla select (può anche essere una `List<>`)
``` c++
      fd_set read_fds, write_fds, except_fds;
      //Forse è meglio con un copy constructor
      demuxtable.convert_to_fd_sets(read_fds,write_fds,except_fds);
```
Adesso che ho preparato il tutto devo poi aspettare realmente gli eventi:
``` c++
      HANDLE max_handle = // Max value in <fd_set>s.
      int result = select
         (max_handle + 1,
         &read_fds, &write_fds, &except_fds,
         timeout);
 
      if (result <= 0)
         throw /* handle error or timeout cases */;
```
Una volta che qualche handle è READY itero sulla lista dei READY 
``` c++
      for (HANDLE h = 0; h <= max_handle; ++h) {
         // This check covers READ_ + ACCEPT_EVENTs
         // because they have the same enum value.
         if (FD_ISSET (&read_fds, h))
            demux_table.table_[h].event_handler_->
               handle_event (h, READ_EVENT);
 
         // … perform the same dispatching logic for
         // WRITE_EVENTs and EXCEPT_EVENTs …
}
```
Non scrivo la parte di registrazioen e remove del enevt handler... Scrivo però la porzione privata della classe:
``` c++
private:
   // Demultiplexing table that maps <HANDLE>s to
   // <Event_Handler>s and <Event_Type>s.
   Demux_Table demux_table_;
};
```
### Capire quanti Reactor ti servono
Semplice??

## Implementare gli event handler

Questo è uno step fondamentale e quello che hare reso nginx così famoso perché è pieno di event handlers HTTP1, HTTP1.1 ... c'è tutto! Quindi per un'app ad alte performance se non hai event handler adeguati sei inutile... Ecco alcune dritte
### Defnire le strategie per integrare app con OS 
Devo far sì che la mia app si integri e parli la lingua del mio OS
1. Hard-coded: definisco a manella l'integrazione fra handle e app. Veloce, sì ma non scalabile...Vedi la plethora di moduli nginx...
2. Generico: bello, elegante ma difficile
### Funzionalità del concrete event handler
è il servizio, la business logic... In uqest aparte posso definire una funzione `handle_event()` magari con 2 parametri (vedi sopra la discussione) on cui gestisci gli eventi (CONNECT, READ, ...)

### Esempio minimo (stessi nomi del codice sopra)
In questa cella creo un mini flusso end-to-end, molto semplice:
1. definisco un `Concrete_Event_Handler` che implementa `handle_event()`;
2. creo un `Simple_Reactor` con `register_handler()` e `handle_events()`;
3. registro `READ_EVENT` e faccio dispatch di un solo evento.


In [7]:
#include <iostream>
// Esempio minimale: un solo handler, un solo event type, un solo dispatch.
class Concrete_Event_Handler : public Event_Handler {
public:
    static constexpr Event_Type READ_EVENT_LOCAL = 01;

    explicit Concrete_Event_Handler(HANDLE handle) : handle_(handle) {}

    void handle_event(HANDLE handle, Event_Type et) override {
        if (et == READ_EVENT_LOCAL) {
            std::cout << "READ_EVENT ricevuto su handle " << handle << std::endl;
        }
    }

    HANDLE get_handle() const override { return handle_; }

private:
    HANDLE handle_;
};


In [8]:

class Simple_Reactor {
public:
    void register_handler(Event_Handler *eh, Event_Type et) {
        event_handler_ = eh;
        event_type_ = et;
    }

    void handle_events() {
        if (event_handler_ != 0) {
            HANDLE handle = event_handler_->get_handle();
            event_handler_->handle_event(handle, event_type_);
        }
    }

private:
    Event_Handler *event_handler_ = 0;
    Event_Type event_type_ = 0;
};


In [9]:
Simple_Reactor reactor;
Concrete_Event_Handler event_handler(3);
reactor.register_handler(&event_handler, Concrete_Event_Handler::READ_EVENT_LOCAL);
reactor.handle_events();


READ_EVENT ricevuto su handle 3
